In [ ]:
from glob import glob
from pathlib import Path
import json
import html

try:
    from dbruntime.display import displayHTML  # type: ignore
except Exception:
    try:
        from IPython.display import display as displayHTML  # type: ignore
    except Exception:
        displayHTML = print  # type: ignore

paths = sorted(glob('../layer_01_bronze/*.json'))
for path in paths:
    settings = json.loads(Path(path).read_text())
    table_name = settings['dst_table_name']
    latest_version = (
        spark.sql(f"DESCRIBE HISTORY {table_name}")
        .selectExpr('max(version) as v')
        .collect()[0]['v']
    )
    json_id = table_name.replace('.', '-') + '-json'
    json_str = json.dumps({'readStreamOptions': {'startingVersion': latest_version}}, indent=4)
    escaped = html.escape(json_str)
    displayHTML(f"""
<h3>{table_name}</h3>
<button onclick="copyJson('{json_id}')">Copy JSON to Clipboard</button>
<textarea id="{json_id}" style="display:none;">{escaped}</textarea>
<pre>{escaped}</pre>
<script>
function copyJson(id) {{
    var textArea = document.getElementById(id);
    textArea.style.display = 'block';
    textArea.select();
    document.execCommand('copy');
    textArea.style.display = 'none';
}}
</script>
""")
